# Tarjetas

dataset: https://docs.google.com/spreadsheets/d/1E2QvH18DyNDw1eGqU40D_KHSEfSzbipV/edit?usp=sharing&ouid=115931210603772521641&rtpof=true&sd=true


In [1]:
import pandas as pd
import numpy
#!pip install openpyxl

In [2]:
%pwd

'C:\\Users\\alejo\\UCEMA\\poo_teorica\\poo_ejercicios\\04-transformaciones'

In [3]:
tarjetas = pd.read_csv('data/tarjetas.xlsx - Sheet1.csv', delimiter=',', index_col='Unnamed: 0')

In [4]:
tarjetas.head()

,PreseTaciOn_Fecha,FeCha_de_Pago,cod_clearing,desc_creaing,NRo_Liquidacion,cod_sucursal,Cod_comercio,Cod_pagadora,Cod_marca_producto,importe_concepto
0,2022-06-15 00:00:00,2022-07-01 00:00:00,1009,VENTAS C/DESCUENTO CONTADO,27822,716,25107406,156,VISA CREDITO,5096.00
1,2022-06-15 00:00:00,2022-07-01 00:00:00,1011,VENTAS C/DTO CUOTAS FINANC. OTORG.,27822,716,25107406,156,VISA CREDITO,413500.60
2,2022-06-15 00:00:00,2022-07-01 00:00:00,2031,ARANCEL,27822,716,25107406,156,VISA CREDITO,7534.70
3,2022-06-15 00:00:00,2022-07-01 00:00:00,2040,PROMO CUOTAS AHORA 12/18,27822,716,25107406,156,VISA CREDITO,18416.85
4,2022-06-15 00:00:00,2022-07-01 00:00:00,2800,TOTAL DEDUCCIONES,27822,716,25107406,156,VISA CREDITO,49585.43


### 1- Pasar todos los nombres de las columnas a minusculas

In [5]:
#cambio el nombre de las columnas y lo guardo en la columna original
tarjetas.columns= tarjetas.columns.str.lower()

In [6]:
tarjetas.columns

Index(['presetacion_fecha', 'fecha_de_pago', 'cod_clearing', 'desc_creaing',
       'nro_liquidacion', 'cod_sucursal', 'cod_comercio', 'cod_pagadora',
       'cod_marca_producto', 'importe_concepto'],
      dtype='object')

### 2-Cuantas sucursales únicas hay? ¿Cuáles?

In [7]:
# metodo unique para obtener los elementos de la columna sucursales sin repeticion
tarjetas.cod_sucursal.unique()

array([716,  98, 671, 519, 241], dtype=int64)

### 3- Cuantas tarjetas de crédito distitas hay? ¿Cuáles?
¿Se te ocurre un nombre mejor para esa columna?

In [8]:
# metodo rename para renombrar, uso el parametro columns para renombrar la columna que se pasa por diccionario, y lo guardo en un nuevo DF
df = tarjetas.rename(columns={'cod_marca_producto':'procesador_de_pago'})

In [ ]:
# uso el metodo unique de nuevo para ver todas las procesadoras de pago, luego uso un ciclo for para imprimir por pantalla los elementos unicos de la columna
procesador = df.procesador_de_pago.unique()
[print(i) for i in procesador]

In [ ]:
# uso el metodo contains para traer de la columna procesador_de_pago todos los elementos que contengan 'CREDITO'
mask_credit_card = df['procesador_de_pago'].str.contains(pat= ' CREDITO')
df.procesador_de_pago[mask_credit_card].unique()


### 4- Cuants filas y columnas hay?


In [ ]:
df.shape

### 5- ¿Cuanto suma el total de importe_concepto?


In [ ]:
total_importe= df.importe_concepto.sum()

print(f'Suma total de importe_concepto: {"{:,}".format(total_importe)}')

### 6- ¿Cuanto dinero se cobro por mes? Cree un lineplot para gráficarlo


In [ ]:
#convierto la columna a datetime, infer_datetime_format te reconoce el formato en que viene la fecha
df['fecha_de_pago'] =  pd.to_datetime(df['fecha_de_pago'], infer_datetime_format=True)

In [ ]:
#hago la suma por mes usando groupby, pero todos los registros son del mismo mes
by_month = df.groupby(df.fecha_de_pago.dt.month)

suma_importe_mes= by_month.sum()['importe_concepto'].to_frame(name='suma mes')
suma_importe_mes

In [ ]:
#creo un grafico de linea con el importe por mes, pero al ser un solo mes no va a devuelve un grafico vacio
suma_importe_mes.plot(kind="line")


In [ ]:
#al ser todos los registros de un mismo mes me parece mas interesante ver el importe por dia
by_day = df.groupby(df.fecha_de_pago.dt.day)

suma_importe_dia= by_day.sum()['importe_concepto'].to_frame(name='suma dia')
suma_importe_dia

In [ ]:
#creo el mismo grafico de linea pero con el importe por dia
plot_importe_dia = suma_importe_dia.plot(kind="line")


### 7- Cuanto dinero se cobro por sucursal?


In [ ]:
# chequeo cuantas sucursales hay
df.cod_sucursal.unique()


In [ ]:
# agrupo por sucursal y hago la suma de la columna importe_concepto
importe_sucursal = df.groupby('cod_sucursal').sum()['importe_concepto'].to_frame(name='importe total')
importe_sucursal

### 8- Cuanto dinero se cobro por mes por sucursal?


In [ ]:
#creo variables con las columnas de fecha por mes y la de sucursal para usarlas en el groupby
month= df.fecha_de_pago.dt.month
sucursal= df.cod_sucursal

importe_x_mes_sucursal= df.groupby(by=[month, sucursal]).sum()['importe_concepto']
importe_x_mes_sucursal

In [ ]:
#repite el paso anterior pero por dia
day= df.fecha_de_pago.dt.day

df.groupby(by=[day, sucursal]).sum()['importe_concepto']


### 9- Cuanto dinero se cobro por mes por sucursal y por descreaing?

In [ ]:
df.columns


In [ ]:
descreaing= df.desc_creaing

df.groupby(by=[month, sucursal, descreaing]).sum()['importe_concepto'].to_frame(name='sum')


### 10- Pase a columnas los valores presentes en desc_creaing y ponga los importes como valores y en el indice la fecha de pago y el codigo de comercio

In [ ]:
#encontre esta manera de hacerlo, usando groupby y unstack a la columna desc_creaing
df.groupby(['fecha_de_pago', 'cod_comercio','desc_creaing'])['importe_concepto'].sum().unstack('desc_creaing')

In [ ]:
#uso el metodo pivot_table para poner en columnas los valores de desc_creaing, como valores los importes y como infices fecha y comercio
pd.pivot_table(df,columns='desc_creaing',values='importe_concepto', index=['fecha_de_pago','cod_comercio'])


### 11- Quedarse solo con tarjeta visa. Crear una columna que sume 'VENTAS C/DESCUENTO CONTADO' con
                                'VENTAS C/DTO CUOTAS FINANC. OTORG.'

In [ ]:
#creo una mascara donde procesador_de_pago sea igual a VISA para filtrar luego en el df
mask_visa = df['procesador_de_pago'].str.contains('VISA')

#uso el metodo pivot_table como en el ejercicio anterior, usando el df filtrado con la mascara
df_solo_visa = pd.pivot_table(df[mask_visa],columns='desc_creaing',values='importe_concepto', index=['fecha_de_pago','cod_comercio'])

#creo una nueva columna a partir de la suma de otras dos
df_solo_visa['ventas con descuento'] = df_solo_visa['VENTAS C/DESCUENTO CONTADO                  '] + df_solo_visa['VENTAS C/DTO CUOTAS FINANC. OTORG.          ']
df_solo_visa['ventas con descuento']

### 12- Quedarse solo con la tarjeta mastercard y crear una columna que sea la suma de 'PERCEPCION IVA R.G. 2408   1,50 %' y  'PERCEPCION IVA R.G. 2408   3,00 %'

In [ ]:
#repito los pasos del ejercicio anterior pero filtrando 'Mastercard'
mask_mastercard = df['procesador_de_pago'].str.contains('MASTERCARD')
df_solo_master = pd.pivot_table(df[mask_mastercard],columns='desc_creaing',values='importe_concepto', index=['fecha_de_pago','cod_comercio'])
df_solo_master['percepcion iva'] = df_solo_master['PERCEPCION IVA R.G. 2408   1,50 %'] + df_solo_master[' PERCEPCION IVA R.G. 2408   3,00 %']
df_solo_master

### Guardar como datasets
a- el dataset de la tarjeta visa
b- el dataset de la tarjeta mastercard
c- cuanto dinero se cobra por mes y por sucursal
d- cuanto se cobra por mes
e- crear un lineplot de cuanto se cobra por mes

In [ ]:
%pwd

In [ ]:
#a
df_solo_visa.to_csv('data/only_visa.csv')
#b
df_solo_master.to_csv('data/only_mastercard.csv')


In [ ]:
#c - importe por mes y sucursal
importe_x_mes_sucursal.to_csv('data/importe_mes_sucursal.csv')
importe_x_mes_sucursal

In [ ]:
#d - importe por mes
suma_importe_mes.to_csv('data/importe_mes.csv')
suma_importe_mes


In [ ]:
from matplotlib import pyplot as plt

In [ ]:
#e - lineplot importe por mes
fig = suma_importe_mes.plot(kind="line")

plt.savefig('data/importe_mes.png')

In [ ]:
#e - lineplot importe por dia
fig = suma_importe_dia.plot(kind="line")

plt.savefig('data/importe_dia.png')